In [433]:
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.model_selection import StratifiedKFold,train_test_split
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OneHotEncoder
from xgboost.sklearn import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from itertools import product
from scipy.spatial import distance
import math
import operator
import re

In [101]:
data_fill=pd.read_csv('fill_rate_data_train.csv')

In [102]:
data_fill.shape

(284674, 12)

In [599]:
data_fill.head()

,unfilled,latitude,longitude,is_couple,city_id,city,gender_request,session_length,special_request,session_local_hour,session_local_day,interval_created_to_session_time,interval,session_period
0,0,33.652760,-117.751945,0,5,Orange County,either,60,0,16,Friday,3170,144,Prime Session
1,0,34.256142,-118.474693,1,2,Los Angeles,female,60,0,10,Friday,1699,144,Early Session
2,0,30.268915,-97.740379,1,12,Austin,both,60,0,17,Thursday,3760,144,Prime Session
3,0,39.039063,-77.361710,0,14,Washington DC,female,60,0,9,Tuesday,2516,144,Early Session
4,0,38.897015,-77.022339,1,14,Washington DC,female,90,0,12,Monday,3445,144,Early Session


In [104]:
data_fill=data_fill.dropna(axis=0,how='any')

In [105]:
data_fill.isnull().sum()

unfilled                            0
latitude                            0
longitude                           0
is_couple                           0
city_id                             0
city                                0
gender_request                      0
session_length                      0
special_request                     0
session_local_hour                  0
session_local_day                   0
interval_created_to_session_time    0
dtype: int64

In [106]:
data_fill.dtypes

unfilled                              int64
latitude                            float64
longitude                           float64
is_couple                             int64
city_id                               int64
city                                 object
gender_request                       object
session_length                        int64
special_request                       int64
session_local_hour                    int64
session_local_day                    object
interval_created_to_session_time      int64
dtype: object

{1: 'San Francisco',
 2: 'Los Angeles',
 3: 'Phoenix/Scottsdale',
 4: 'Miami',
 5: 'Orange County',
 9: 'Silicon Valley',
 10: 'NYC',
 12: 'Austin',
 13: 'San Diego',
 14: 'Washington DC',
 15: 'Seattle',
 16: 'Vancouver',
 17: 'Dallas',
 18: 'Chicago',
 20: 'Boston',
 21: 'Atlanta',
 22: 'Philadelphia',
 23: 'London',
 24: 'Minneapolis',
 25: 'Houston',
 26: 'Toronto',
 27: 'Indianapolis',
 28: 'Denver',
 29: 'Portland',
 30: 'Northern New Jersey',
 31: 'Las Vegas',
 32: 'Inland Empire',
 33: 'Memphis',
 34: 'Orlando',
 35: 'Honolulu',
 36: 'Long Island/The Hamptons',
 37: 'Oklahoma City',
 38: 'Salt Lake City/Park City',
 39: 'Charlotte',
 40: 'St Louis',
 41: 'Baltimore',
 42: 'Cincinnati',
 43: 'Norfolk/Virginia Beach',
 44: 'Milwaukee',
 45: 'San Antonio',
 46: 'New Orleans',
 48: 'Cleveland',
 49: 'Louisville',
 50: 'Santa Barbara',
 51: 'Napa/Sonoma County',
 52: 'Nashville',
 53: 'Oakland/East Bay',
 54: 'Sydney',
 55: 'Columbus',
 56: 'NYC - The Bronx',
 57: 'NYC - Queens',
 5

In [107]:
data_fill.head()

,unfilled,latitude,longitude,is_couple,city_id,city,gender_request,session_length,special_request,session_local_hour,session_local_day,interval_created_to_session_time
0,0,33.652760,-117.751945,0,5,Orange County,either,60,0,16,Friday,3170
1,0,34.256142,-118.474693,1,2,Los Angeles,female,60,0,10,Friday,1699
2,0,30.268915,-97.740379,1,12,Austin,both,60,0,17,Thursday,3760
3,0,39.039063,-77.361710,0,14,Washington DC,female,60,0,9,Tuesday,2516
4,0,38.897015,-77.022339,1,14,Washington DC,female,90,0,12,Monday,3445


In [ ]:
def geodata_generate(row,df):
    latitude=list(df.latitude.quantile([0.25,0.5,0.75]).values)
    longitude=list(df.longitude.quantile([0.25,0.5,0.75]).values)
    #distance=[]
    for n,(lat,long) in zip(range(9),list(product(latitude,longitude))):
            #print (lat,long)
            #print(row['latitude'],row['longitude'])
            #row['geo_'+str(n)]=euclidean_dist(row['latitude'],row['longitude'],lat,long)
            row['geo_'+str(n)]=distance.euclidean((row['latitude'],row['longitude']),(lat,long))
    return row   

In [588]:
## transformation
def dummie_transformation(df,cat_list):
    df_dummies=pd.get_dummies(df,columns=cat_list,prefix=cat_list,drop_first=True)
    #df_dummies.drop(cat_list,axis=1,inplace=True)
    return df_dummies

def cat_session_hour(x):
    if 8<=x<16:
        return 'Early Session'
    elif 16<=x<20:
        return 'Prime Session'
    else:
        return 'Late Session'

    
def truncate_interval(x):
    if x >144:
        return 144.0
    elif x< 0 :
        return 0.0
    else:
        return x 
    
    
def geodata_generate(x_train,x_test):
    latitude=list(x_train.latitude.quantile([0.25,0.5,0.75]).values)
    longitude=list(x_train.longitude.quantile([0.25,0.5,0.75]).values)
    for n,(lat,long) in zip(range(9),list(product(latitude,longitude))):
        x_train['geo_'+str(n)]=euclidean_dist(x_train.latitude,x_train.longitude,lat,long)
        x_test['geo_'+str(n)]=euclidean_dist(x_test.latitude,x_test.longitude,lat,long)
        
    trainset=x_train.drop(['city_id','city','latitude','longitude'],axis=1)
    testset=x_test.drop(['city_id','city','latitude','longitude'],axis=1)
    return trainset,testset
     


def euclidean_dist(x1,y1,x2,y2):
    return round(np.sqrt(pow((x1-x2),2)+pow((y1-y2),2)),6)
    

def scaler(x_train,x_test):
    g=x_train.columns.to_series().groupby(x_train.dtypes).groups
    df_type={k.name: v for k,v in g.items()}
    scale_list=list(df_type['int64'])+list(df_type['float64'])
    #print (scale_list)
    sc=MinMaxScaler()
    x_train_scaled=x_train.loc[:,[s for s in x_train.columns if s not in scale_list]]
    x_test_scaled=x_test.loc[:,[s for s in x_test.columns if s not in scale_list]]
    for column in scale_list:
        x_train_scaled[column]=sc.fit_transform(x_train.loc[:,column])
        x_test_scaled[column]=sc.transform(x_test.loc[:,column])
    return x_train_scaled,x_test_scaled

def math_transform(df):
    g=df.columns.to_series().groupby(df.dtypes).groups
    df_type={k.name: v for k,v in g.items()}
    trans_list=list(df_type['int64'])+list(df_type['float64'])
    trans_list=[s for s in trans_list if s
                not in ['unfilled','is_couple','special_request']]
    #print (trans_list)
    df_trans=df.loc[:,[c for c in df.columns if c not in trans_list]]
    adjust=1e-10
    for column in trans_list: 
        df_trans[column+'_log']=np.log(df[column]+adjust)
        df_trans[column+'_inv']=1/(df[column]+adjust)
        df_trans[column+'_sqrt']=np.sqrt(df[column])
        df_trans[column+'_pow']=pow(df[column],2)
    return df_trans

def auc_cross_validation(model,X_train,y_train,city_id,city_dict):
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    skf=StratifiedKFold(n_splits=5,random_state=123,shuffle=False)
    score=[]
    for train_ind,test_ind in skf.split(X_train,y_train):
        train_data_x,test_data_x=X_train[train_ind],X_train[test_ind]
        train_data_y,test_data_y=y_train[train_ind],y_train[test_ind]
        
        model.fit(train_data_x,train_data_y)
        pred=model.predict_proba(test_data_x)[:,1]
        score.append(metrics.roc_auc_score(test_data_y,pred))
    #print ("Model for %s is: %0.5f"%(city_dict[city_id],np.mean(score)))
    return np.mean(score)
        
    
def stringfy(model):
    model_name=model.__class__.__name__
    return re.sub(r'[a-z]','',model_name)

def save_city_model(model,city_id):
    model_name=stringfy(model)
    #file_name=model_name+'_'+city_dict[city_id]+'.pkl'
    file_name=model_name+'_'+re.sub(r'/','_',city_dict[city_id])+'.pkl'
    save_path=os.getcwd()+'/'+file_name
    #print(save_path)
    if os.path.exists(save_path):
        os.remove(save_path)
        print('Old model is removed')
    try:
        outfile=open(save_path,'wb')
        pickle.dump(model,outfile)
        #print(1)
        outfile.close()
        print('Saved the Model as ',file_name)
    except:
        print('Failed to Save the Model')

def show_current_model(**path):
    if path:
        pass
    else:
        path=os.getcwd()
    try:
        for file in os.listdir(path):
            if file.endswith('.pkl'):
                print(file)
    except:
        print('Invalid Path')

def load_model(city_id,**path):
    if path:
        pass
    else:
        path=os.getcwd()
    #file_end='_'+city_dict[city_id]+'.pkl'
    file_end='_'+re.sub(r'/','_',city_dict[city_id])+'.pkl'
    #print(path)
    #print(file_end)
    try:
        for file in os.listdir(path):
            if file.endswith(file_end):
                file_path=path+'/'+file
                infile=open(file_path,'rb')
                model=pickle.load(infile)
                infile.close()
                print ('%s Loaded'%file)
                return model
        if not model:
            print('Model Not Exist.')
    except:
        print('Invalid Path')
        
def save_threshold(threshold_dict,**path):
    if path:
        save_path=path
    else:
        save_path=os.getcwd()+'/threshold.pkl'
    if os.path.exists(save_path):
        os.remove(save_path)
        print('Old threshold file removed.')
    try:    
        outfile=open(save_path,'wb')
        pickle.dump(threshold_dict,outfile)
        outfile.close()
        print('Successfully Saved Threshold.')
    except:
        print('Failed to Save Threshold.')
        
def load_threshold(city_id,**path):
    if path:
        save_path=path
    else:
        save_path=os.getcwd()+'/threshold.pkl'
    try:
        infile=open(save_path,'rb')
        thres_dict=pickle.load(infile)
        infile.close()
        print ('Threshold for %s has been loaded.'%city_dict[city_id])
        return thres_dict[city_id]
        
    except:
        print('Failed to load threshold')
        
        
    
    
    
    
    
    
    
def best_threshold_cv(model,X_train,y_train):
    X_train=np.array(X_train)
    y_train=np.array(y_train)
    skf=StratifiedKFold(n_splits=3,random_state=1234,shuffle=False)
    best_f1_score=-1
    best_recall=0
    best_threshold=0
    best_accuracy=0
    
    for threshold in np.arange(0.001,0.999,0.001):
        f1_score=[]
        accuracy_score=[]
        recall_score=[]
        for train_ind,test_ind in skf.split(X_train,y_train):
            test_data_x=X_train[test_ind]
            test_data_y=y_train[test_ind]
            p1=np.where(model.predict_proba(test_data_x)[:,1]>=threshold,1,0)
            f1_score.append(metrics.f1_score(test_data_y,p1))
            recall_score.append(metrics.recall_score(test_data_y,p1))
            accuracy_score.append(metrics.accuracy_score(test_data_y,p1))
        current_f1_score=np.mean(f1_score)
        current_recall_score=np.mean(recall_score)
        current_accuracy_score=np.mean(accuracy_score)
        if (current_f1_score>best_f1_score):
            best_recall=current_recall_score
            best_f1_score=current_f1_score
            best_accuracy=current_accuracy_score
            best_threshold=threshold
    #print('Best Threshold:%.6f, Best Accuracy:%.6f, Best F1 Score:%.6f, Best Recall: %.6f'%(best_threshold,best_accuracy,best_f1_score,best_recall))
    return best_threshold,best_accuracy,best_f1_score,best_recall
    

    

In [555]:
save_threshold({1:'fuck'})

Old threshold file removed.
Successfully Saved Threshold.


In [556]:
load_threshold(city_id=1)

Threshold for San Francisco has been loaded.


'fuck'

In [422]:
a_dict=dict()
a_dict[stringfy(clf)]=auc_cross_validation(LogisticRegression(),X_train_scaled,y_train,1,city_dict)

Model for San Francisco is: 0.74358


In [432]:
a={'a':1,'b':2,'c':3}
a.get(sorted(a.items(),key=operator.itemgetter(1),reverse=True)[0][0],None)

3

In [436]:
model_list=[LogisticRegression(),XGBClassifier()]
a=dict(zip([stringfy(n) for n in model_list ],model_list))

In [438]:
a.get('LR')

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
data_fill['interval']=data_fill.interval_created_to_session_time.apply(truncate_interval)
data_fill['session_period']=data_fill.session_local_hour.apply(cat_session_hour)

##cat list
cat_list=['session_length','session_local_day','gender_request','session_period']
data_fill_dummies=dummie_transformation(df=data_fill,cat_list=cat_list)
## drop list
drop_list=['interval_created_to_session_time','session_local_hour']
data_fill_dummies.drop(drop_list,axis=1,inplace=True)

In [320]:
data_fill_dummies_add_geo.head()

,unfilled,is_couple,special_request,interval,session_length_90,session_length_120,session_length_150,session_local_day_Monday,session_local_day_Saturday,session_local_day_Sunday,...,session_period_Prime Session,geo_0,geo_1,geo_2,geo_3,geo_4,geo_5,geo_6,geo_7,geo_8
13,1,0,0,144,1,0,0,0,1,0,...,1,0.028166,0.039177,0.053501,0.015921,0.031544,0.048193,0.015184,0.031179,0.047955
15,0,0,0,144,0,0,0,0,0,0,...,0,0.178456,0.163307,0.148173,0.172505,0.156783,0.140950,0.168783,0.152677,0.136368
20,1,1,0,144,0,0,0,0,0,0,...,1,0.049524,0.034364,0.021293,0.046547,0.029915,0.012938,0.048458,0.032810,0.018681
32,0,0,0,144,1,0,0,0,0,0,...,1,0.067582,0.083924,0.100703,0.072950,0.088304,0.104381,0.079633,0.093901,0.109157
34,0,1,0,144,1,0,0,0,0,0,...,0,0.042007,0.046859,0.056625,0.025349,0.032768,0.045656,0.012275,0.024122,0.039907


In [598]:
load_threshold(1)

Threshold for San Francisco has been loaded.


{'best_accuracy': 0.82255061095393744,
 'best_f1_score': 0.45224855975289668,
 'best_recall': 0.60398751417902419,
 'best_threshold': 0.17899999999999999}

In [595]:
import warnings
warnings.filterwarnings("ignore")

city_dict=dict(list(set(zip(data_fill.city_id,data_fill.city))))
city_list=list(city_dict.keys())[:1]
model_list=[LogisticRegression(),XGBClassifier()]
model_list_abb=dict(zip([stringfy(n) for n in model_list ],model_list))
failed_city=[]
threshold_dict=dict()
n=0
#data_fill_dummies_addgeo=
for city_id in city_list:
    #data_fill_dummies_add_geo=geodata_generate(data_fill_dummies[data_fill_dummies.city_id==city_id])
    #data_fill_dummies_add_geo=math_transform(data_fill_dummies_add_geo)
    data_fill_dummies_city=data_fill_dummies[data_fill_dummies.city_id==city_id]
    predictors=[s for s in data_fill_dummies_city if s!='unfilled']
    X_Train=data_fill_dummies_city.loc[:,predictors]
    y_Train=data_fill_dummies_city.loc[:,'unfilled']
    X_train,X_test,y_train,y_test=train_test_split(X_Train,y_Train,test_size=0.2,random_state=123)
    X_train_add_geo,X_test_add_geo=geodata_generate(X_train,X_test)
    X_train_add_geo_trans=math_transform(X_train_add_geo)
    X_test_add_geo_trans=math_transform(X_test_add_geo)
    X_train_scaled,X_test_scaled=scaler(X_train_add_geo_trans,X_test_add_geo_trans)   
    auc_dict=dict()
    try:
        for model in model_list:
            auc_dict[stringfy(model)]=auc_cross_validation(model,X_train_scaled,y_train,city_id,city_dict)
            
        print ('Best Model for %s: '%city_dict.get(city_id),sorted(auc_dict.items(),key=operator.itemgetter(1),reverse=True)[0][0],'| Best Auc:',sorted(auc_dict.items(),key=operator.itemgetter(1),reverse=True)[0][1] )
        best_model=model_list_abb.get(sorted(auc_dict.items(),key=operator.itemgetter(1),reverse=True)[0][0],None)
        best_model.fit(np.array(X_train_scaled),np.array(y_train))
        best_threshold,best_accuracy,best_f1_score,best_recall=best_threshold_cv(best_model,X_train_scaled,y_train)
        threshold_dict[city_id]={'best_threshold':best_threshold,'best_accuracy':best_accuracy,'best_f1_score':best_f1_score,'best_recall':best_recall}
        save_city_model(best_model,city_id)
    except:
            print('Failed to build model for %s Sample size too small.'%city_dict[city_id])
            failed_city.append(city_dict[city_id])
    

    n+=1
    if n==len(city_list):
        
        print('''
        ###############################
              Training Finished
        ###############################
        
        ''')
        save_threshold(threshold_dict)
    


Best Model for San Francisco:  XGBC | Best Auc: 0.773629913544
Old model is removed
Saved the Model as  XGBC_San Francisco.pkl

        ###############################
              Training Finished
        ###############################
        
        
Old threshold file removed.
Successfully Saved Threshold.


In [564]:
best_threshold,best_accuracy,best_f1_score,best_recall=best_threshold_cv(best_model,X_train_scaled,y_train)

In [569]:
a={}
a[1]={'best_threshold':best_threshold,'best_accuracy':best_accuracy,'best_f1_score':best_f1_score,'best_recall':best_recall}

In [578]:
load_threshold(2)

Threshold for Los Angeles has been loaded.


{'best_accuracy': 0.88197681070642153,
 'best_f1_score': 0.25399614732500958,
 'best_recall': 0.36181229773462781,
 'best_threshold': 0.106}

In [533]:
failed_city

['Temecula']

In [361]:
clf=LogisticRegression()
clf.fit(X_train_scaled,y_train)
pred=clf.predict_proba(X_test_scaled)[:,1]
metrics.roc_auc_score(y_test,pred)

0.7518563129510667

In [348]:
X_train_scaled.head()

,session_length_90,session_length_120,session_length_150,session_local_day_Monday,session_local_day_Saturday,session_local_day_Sunday,session_local_day_Thursday,session_local_day_Tuesday,session_local_day_Wednesday,gender_request_either,...,geo_6_sqrt,geo_6_sin,geo_7_log,geo_7_inv,geo_7_sqrt,geo_7_sin,geo_8_log,geo_8_inv,geo_8_sqrt,geo_8_sin
90431,0,0,0,0,0,1,0,0,0,1,...,0.023528,0.551207,0.365994,0.012765,0.020360,0.548722,0.450791,0.002203,0.022971,0.551566
33259,0,0,0,0,1,0,0,0,0,1,...,0.029952,0.560954,0.390743,0.009503,0.024006,0.553437,0.414900,0.003586,0.017762,0.545674
43236,0,0,0,0,0,0,0,0,1,0,...,0.023218,0.550791,0.382238,0.010517,0.022692,0.551658,0.473379,0.001621,0.026964,0.557045
202541,1,0,0,1,0,0,0,0,0,0,...,0.022806,0.550245,0.320977,0.021829,0.014960,0.543013,0.346198,0.009111,0.010722,0.539981
35703,0,0,0,0,0,0,0,0,0,0,...,0.060850,0.637217,0.528833,0.001828,0.057960,0.630200,0.578949,0.000386,0.056376,0.622917


In [350]:
X_train.columns

Index(['is_couple', 'special_request', 'session_length_90',
       'session_length_120', 'session_length_150', 'session_local_day_Monday',
       'session_local_day_Saturday', 'session_local_day_Sunday',
       'session_local_day_Thursday', 'session_local_day_Tuesday',
       'session_local_day_Wednesday', 'gender_request_either',
       'gender_request_female', 'gender_request_male',
       'session_period_Late Session', 'session_period_Prime Session',
       'interval_log', 'interval_inv', 'interval_sqrt', 'interval_sin',
       'geo_0_log', 'geo_0_inv', 'geo_0_sqrt', 'geo_0_sin', 'geo_1_log',
       'geo_1_inv', 'geo_1_sqrt', 'geo_1_sin', 'geo_2_log', 'geo_2_inv',
       'geo_2_sqrt', 'geo_2_sin', 'geo_3_log', 'geo_3_inv', 'geo_3_sqrt',
       'geo_3_sin', 'geo_4_log', 'geo_4_inv', 'geo_4_sqrt', 'geo_4_sin',
       'geo_5_log', 'geo_5_inv', 'geo_5_sqrt', 'geo_5_sin', 'geo_6_log',
       'geo_6_inv', 'geo_6_sqrt', 'geo_6_sin', 'geo_7_log', 'geo_7_inv',
       'geo_7_sqrt', 'geo_7_sin

In [109]:
data_fill['interval']=data_fill.interval_created_to_session_time.apply(truncate_interval)
data_fill['session_period']=data_fill.session_local_hour.apply(cat_session_hour)

##cat list
cat_list=['session_length','session_local_day','gender_request','session_period']
data_fill_dummies=dummie_transformation(df=data_fill,cat_list=cat_list)
## drop list
drop_list=['interval_created_to_session_time','session_local_hour']
data_fill_dummies.drop(drop_list,axis=1,inplace=True)


In [110]:
data_fill_dummies.head()

,unfilled,latitude,longitude,is_couple,city_id,city,special_request,session_local_hour,interval_created_to_session_time,interval,...,session_local_day_Saturday,session_local_day_Sunday,session_local_day_Thursday,session_local_day_Tuesday,session_local_day_Wednesday,gender_request_either,gender_request_female,gender_request_male,session_period_Late Session,session_period_Prime Session
0,0,33.652760,-117.751945,0,5,Orange County,0,16,3170,144,...,0,0,0,0,0,1,0,0,0,1
1,0,34.256142,-118.474693,1,2,Los Angeles,0,10,1699,144,...,0,0,0,0,0,0,1,0,0,0
2,0,30.268915,-97.740379,1,12,Austin,0,17,3760,144,...,0,0,1,0,0,0,0,0,0,1
3,0,39.039063,-77.361710,0,14,Washington DC,0,9,2516,144,...,0,0,0,1,0,0,1,0,0,0
4,0,38.897015,-77.022339,1,14,Washington DC,0,12,3445,144,...,0,0,0,0,0,0,1,0,0,0


In [522]:
re.sub(r'/','_','Phoenix/Scottsdale')

'Phoenix_Scottsdale'

In [113]:
 data_fill_dummies.columns

Index(['unfilled', 'latitude', 'longitude', 'is_couple', 'city_id', 'city',
       'special_request', 'interval', 'session_length_90',
       'session_length_120', 'session_length_150', 'session_local_day_Monday',
       'session_local_day_Saturday', 'session_local_day_Sunday',
       'session_local_day_Thursday', 'session_local_day_Tuesday',
       'session_local_day_Wednesday', 'gender_request_either',
       'gender_request_female', 'gender_request_male',
       'session_period_Late Session', 'session_period_Prime Session'],
      dtype='object')

In [606]:
#categorize the feature

def cat_preprocessing(df):
    cat_list=['session_length','session_local_day','gender_request','session_period']
    drop_list=['interval_created_to_session_time','session_local_hour']
    df_trans=df.dropna(axis=0,how='any')
    df_trans['interval']=df_trans.interval_created_to_session_time.apply(truncate_interval)
    df_trans['session_period']=df_trans.session_local_hour.apply(cat_session_hour)
    df_trans_dummies=dummie_transformation(df=df_trans,cat_list=cat_list)
    df_trans_dummies.drop(drop_list,axis=1,inplace=True)
    
    return df_trans_dummies
    
    

In [607]:
a=cat_preprocessing(data_fill)

In [608]:
a.columns

Index(['unfilled', 'latitude', 'longitude', 'is_couple', 'city_id', 'city',
       'special_request', 'interval', 'session_length_90',
       'session_length_120', 'session_length_150', 'session_local_day_Monday',
       'session_local_day_Saturday', 'session_local_day_Sunday',
       'session_local_day_Thursday', 'session_local_day_Tuesday',
       'session_local_day_Wednesday', 'gender_request_either',
       'gender_request_female', 'gender_request_male',
       'session_period_Late Session', 'session_period_Prime Session'],
      dtype='object')

In [604]:
data_fill_dummies.columns

Index(['unfilled', 'latitude', 'longitude', 'is_couple', 'city_id', 'city',
       'special_request', 'interval', 'session_length_90',
       'session_length_120', 'session_length_150', 'session_local_day_Monday',
       'session_local_day_Saturday', 'session_local_day_Sunday',
       'session_local_day_Thursday', 'session_local_day_Tuesday',
       'session_local_day_Wednesday', 'gender_request_either',
       'gender_request_female', 'gender_request_male',
       'session_period_Late Session', 'session_period_Prime Session'],
      dtype='object')

In [ ]:
import numpy as np